# INFO 4271 - Group Project

Issued: June 11, 2024

Due: July 22, 2024

Please submit a link to your code base (ideally with a branch that does not change anymore after the submission deadline) and your 4-page report via email to carsten.eickhoff@uni-tuebingen.de by the due date. One submission per team.

---

# 1. Web Crawling & Indexing
Crawl the web to discover **English content related to Tübingen**. The crawled content should be stored locally. If interrupted, your crawler should be able to re-start and pick up the crawling process at any time.

In [5]:
import requests
from bs4 import BeautifulSoup
import justext
from boilerpy3 import extractors
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag
import re
from trafilatura import fetch_url, extract
import contractions
from nltk.stem import WordNetLemmatizer
#Add a document to the index. You need (at least) two parameters:
	#doc: The document to be indexed.
	#index: The location of the local index storing the discovered documents.
def index(doc, index):
    #TODO: Implement me
	pass

#Crawl the web. You need (at least) two parameters:
	#frontier: The frontier of known URLs to crawl. You will initially populate this with your seed set of URLs and later maintain all discovered (but not yet crawled) URLs here.
	#index: The location of the local index storing the discovered documents. 
def crawl(frontier, index):
    #TODO: Implement me
	pass

# 2. Query Processing 
Process a textual query and return the 100 most relevant documents from your index. Please incorporate **at least one retrieval model innovation** that goes beyond BM25 or TF-IDF. Please allow for queries to be entered either individually in an interactive user interface (see also #3 below), or via a batch file containing multiple queries at once. The batch file will be formatted to have one query per line, listing the query number, and query text as tab-separated entries. An example of the batch file for the first two queries looks like this:

```
1   tübingen attractions
2   food and drinks
```

In [72]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
import re

# Process query
# remove stop words, lemmatize, etc.
# Todo: Sonderzeichen, bindestrich wörter aufteilen
def prepare_query(query):
	query_tokenized= []
	stop_words = set(stopwords.words('english'))
	lemmatizer = WordNetLemmatizer()
	query_lower = query.lower()
 	
	for term, tag in pos_tag(word_tokenize(query_lower)):
		if re.compile(r'[^a-zA-Z0-9äöüÄÖÜ\s]').match(term):
			continue
		if term in stop_words:
			continue

		# split words with "-"
		if '-' in term:
			t = term.split('-')
			for i, tag in pos_tag(t):
				ltag = tag[0].lower()
				if ltag in ['a', 'r', 'n', 'v']:
					i = lemmatizer.lemmatize(i, ltag)	
					if i not in query_tokenized:
						query_tokenized.append(i)
		else: # words without "-"
			ltag = tag[0].lower()
			if ltag in ['a', 'r', 'n', 'v']:
				term = lemmatizer.lemmatize(term, ltag)	
				if term not in query_tokenized:
					query_tokenized.append(term)
	return query_tokenized

#get all the documents that include all the queryterms from our index 
def get_documents_from_index(query_tokenized, index):
	documents = []
	first_document = True
	second_document = False

	for term in query_tokenized:
		# first term in query --> get all documents amtching this term from index
		if term in index.keys() and first_document:   		
			first_document = False	
			documents = index[term]
			second_document = True
		# check second term. get documents from first iteration which have this term as well. use skip pointers for both lists 
		elif term in index.keys() and second_document: 
			second_document = False
			matches = []
			new_term_list = index[term]
			i = 0
			j = 0
			while i < len(documents) and j < len(new_term_list):  
				if documents[i][0] == new_term_list[j][0]:
					matches.append(documents[i])							# append entry including skip pointers for future iterations 
					i += 1
					j += 1
				elif documents[i][0] < new_term_list[j][0]:                 # A < B
					if documents[i][1]:                                     # If there is a skip pointer
						if documents[i][1][1] <= new_term_list[j][0]:       # Take it if it does not carry you beyond the id pointed to by B
							i = documents[i][1][0]
						else:                                       		# Otherwise increase the pointer by 1
							i += 1
					else:
						i += 1
				elif new_term_list[j][1]:                                   # If there is a skip pointer
					if new_term_list[j][1][1] <= documents[i][0]:           # Take it if it does not carry you beyond the id pointed to by A
						j = new_term_list[j][1][0]
					else:
						j += 1                                     			# Otherwise increase the pointer by 1
				else:
					j += 1 
			documents = matches		
			
		# check current term. get documents from previous iteration which have this term as well. use skip pointers only for list in index.
		elif term in index.keys():
			matches = []
			new_term_list = index[term]
			i = 0
			j = 0
			while i < len(documents) and j < len(new_term_list):  
				if documents[i][0] == new_term_list[j][0]:
					matches.append(documents[i])							# append entry including skip pointers for future iterations 
					i += 1
					j += 1
				elif documents[i][0] < new_term_list[j][0]:    				# A < B
					i += 1													# This time no skip pointers, because they aren't correct anymore

				elif new_term_list[j][1]:                                   # If there is a skip pointer
					if new_term_list[j][1][1] <= documents[i][0]:           # Take it if it does not carry you beyond the id pointed to by A
						j = new_term_list[j][1][0]
					else:
						j += 1                                     			# Otherwise increase the pointer by 1
				else:
					j += 1 
			documents = matches		
		else:
			documents = []
			break															# query term wasn't found, no document can satisfy query

		print(documents)	

	# get rid of skip pointers:
	tmp = []
	for element in documents:
		tmp.append(element[0])
	documents = tmp

	# old code: remove later
	#for term in query_tokenized:
	#	documents_term = []
	#	if term in index.keys():
	#		set_first_document = True
	#		for document_entry in index[term]:
	#			documents_term.append(document_entry[0])
	#	if first_document:
	#		documents = documents_term
	#	else:
	#		documents = list(set(documents).intersection(documents_term))
	#	if (set_first_document):
	#		first_document = False # Todo: anders lösen
	return documents

#Retrieve documents relevnt to a query. You need (at least) two parameters:
	#query: The user's search query
	#index: The location of the local index storing the discovered documents.
def retrieve(query, index):
    #TODO: Implement me

	# get processed query
	query_tokenized= prepare_query(query)		

	#get dictionary from indes
	document_index = index[0]

	# get all the documents, that include all the terms from our query
	documents = get_documents_from_index(query_tokenized, document_index)
	
	print('erhaltene Dokumente', documents)

	# Todo: implement retreival model
	# Use generative model, query likelihood model?
	# Todo: implemenrt ranking


	# diversify: use hash
	hashes = index[2]
	# Todo: implement diversify function

#[['D0', [2, 'D2']], ['D1', None], ['D2', None], ['D14', None]]
test_index = ({'tübingen': [[0,[2,2]],[1,None],[2,None], [14,None]],
			 'henry': [[0,None],[1,None],[2,None],[12,None],[13,None], [14,None]],
			 'attraction': [[0,[2,12]],[3,[3,14]],[12,None],[14,None]]},
			 [['URL0'],['URL1'],['URL2']],
			 ['Hash1','Hash1'])
retrieve('henry, tübingen-attractions' , test_index)

['tübingen', 'attractions']
tübingen
attraction
[[0, None], [1, None], [2, None], [12, None], [13, None], [14, None]]
[[0, None], [1, None], [2, None], [14, None]]
[[0, None], [14, None]]
erhaltene Dokumente [0, 14]


# 3. Search Result Presentation
Once you have a result set, we want to return it to the searcher in two ways: a) in an interactive user interface. For this user interface, please think of **at least one innovation** that goes beyond the traditional 10-blue-links interface that most commercial search engines employ. b) as a text file used for batch performance evaluation. The text file should be formatted to produce one ranked result per line, listing the query number, rank position, document URL and relevance score as tab-separated entries. An example of the first three lines of such a text file looks like this:

```
1   1   https://www.tuebingen.de/en/3521.html   0.725
1   2   https://www.komoot.com/guide/355570/castles-in-tuebingen-district   0.671
1   3   https://www.unimuseum.uni-tuebingen.de/en/museum-at-hohentuebingen-castle   0.529
...
1   100 https://www.tuebingen.de/en/3536.html   0.178
2   1   https://www.tuebingen.de/en/3773.html   0.956
2   2   https://www.tuebingen.de/en/4456.html   0.797
...
```

In [1]:
#TODO: Implement an interactive user interface for part a of this exercise.

#Produce a text file with 100 results per query in the format specified above.
def batch(results):
    #TODO: Implement me.    
    pass

# 4. Performance Evaluation 
We will evaluate the performance of our search systems on the basis of five queries. Two of them are avilable to you now for engineering purposes:
- `tübingen attractions`
- `food and drinks`

The remaining three queries will be given to you during our final session on July 23rd. Please be prepared to run your systems and produce a single result file for all five queries live in class. That means you should aim for processing times of no more than ~1 minute per query. We will ask you to send carsten.eickhoff@uni-tuebingen.de that file.

# Grading
Your final projects will be graded along the following criteria:
- 25% Code correctness and quality (to be delivered on this sheet)
- 25% Report (4 pages, PDF, explanation and justification of your design choices)
- 25% System performance (based on how well your system performs on the 5 queries relative to the other teams in terms of nDCG)
- 15% Creativity and innovativeness of your approach (in particular with respect to your search system #2 and user interface #3 innovations)
- 10% Presentation quality and clarity

# Permissible libraries
You can use any general-puprose ML and NLP libraries such as scipy, numpy, scikit-learn, spacy, nltk, but please stay away from dedicated web crawling or search engine toolkits such as scrapy, whoosh, lucene, terrier, galago and the likes. Pretrained models are fine to use as part of your system, as long as they have not been built/trained for retrieval. 
